<a href="https://colab.research.google.com/github/schliffen/QuantResearch/blob/master/Backtest_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>